In [2]:
import numpy as np
from scipy.optimize import minimize

# SABR Volatility function
def sabr_volatility(F, K, T, alpha, beta, rho, nu):
    """
    Calculate the SABR implied volatility.

    Parameters:
    F (float): Forward rate
    K (float): Strike price
    T (float): Time to maturity
    alpha (float): Volatility of the underlying asset at the start
    beta (float): Elasticity parameter of the volatility
    rho (float): Correlation between the asset price and its volatility
    nu (float): Volatility of volatility

    Returns:
    float: Implied volatility estimated by the SABR model
    """
    if F == K:
        fk_beta = F**(1 - beta)
    else:
        fk_beta = (F * K)**((1 - beta) / 2)

    z = nu / alpha * fk_beta * np.log(F / K)
    x_z = np.log((np.sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
    sigma = alpha * (1 + ((1 - beta)**2 * alpha**2 / (24 * fk_beta**2) + 0.25 * rho * beta * nu * alpha / fk_beta + (2 - 3 * rho**2) * nu**2 / 24) * T) * z / x_z
    return sigma

# Objective function for the SABR model calibration
def sabr_objective(params, market_vols, strikes, forward, expiry):
    """
    Objective function to be minimized during the calibration of SABR parameters.

    Parameters:
    params (tuple): Tuple of SABR parameters (alpha, beta, rho, nu)
    market_vols (array): Array of market implied volatilities
    strikes (array): Array of strike prices
    forward (float): Forward rate
    expiry (float): Time to maturity

    Returns:
    float: Sum of squared errors between model and market volatilities
    """
    alpha, beta, rho, nu = params
    errors = 0
    for i, K in enumerate(strikes):
        model_vol = sabr_volatility(forward, K, expiry, alpha, beta, rho, nu)
        errors += (model_vol - market_vols[i])**2
    return errors

# Example data
market_vols = np.array([0.20, 0.22, 0.25])  # Example market volatilities
strikes = np.array([100, 105, 110])         # Corresponding strike prices
forward_rate = 100                          # Example forward rate
expiry_time = 1                             # Time to maturity in years

# Initial guesses for SABR parameters
initial_params = [0.2, 0.5, 0, 0.2]  # Initial guesses: alpha, beta, rho, nu

# Calibration using scipy.optimize.minimize
result = minimize(sabr_objective, initial_params, args=(market_vols, strikes, forward_rate, expiry_time),
                  bounds=[(0.001, None), (0, 1), (-1, 1), (0.001, None)])

result.x  # This will output the optimized parameters: alpha, beta, rho, nu



/var/folders/_b/zgcv3szx52v_rfyrvvjcb6y00000gn/T/ipykernel_18588/3502696033.py:24: RuntimeWarning: invalid value encountered in scalar power
  fk_beta = (F * K)**((1 - beta) / 2)
/var/folders/_b/zgcv3szx52v_rfyrvvjcb6y00000gn/T/ipykernel_18588/3502696033.py:26: RuntimeWarning: invalid value encountered in log
  z = nu / alpha * fk_beta * np.log(F / K)


array([0.2, 0.5, 0. , 0.2])

In [8]:
sigmas = np.array([0.4364, 0.4191, 0.41706,0.41605, 0.41619,0.41757,0.42003,0.42370,0.42847,0.43428,0.46603])

strikes_in_bps = np.array([200,100,75,50,25,10,25,50,75,100,200])
t_exp = 10
tenor = 30
atm_sigma = 0.41757
f = 0.23401

In [9]:
result = minimize(sabr_objective, initial_params, args=(sigmas, strikes_in_bps, f, t_exp),
                  bounds=[(0.001, None), (0, 1), (-1, 1), (0.001, None)])

result.x  # This will output the optimized parameters: alpha, beta, rho, nu

array([ 0.41402381,  0.23858889, -0.1373466 ,  0.01453566])

In [21]:
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


def SABR_market_vol(K,f,t_exp,alpha,beta,nu,rho):
    '''Given a list of strike prices and SABR parameters, returns what the SABR
    model predicts the market volatility to be. Calculated from equations (2.17) 
    and (2.18) in Hagan, Patrick S., Deep Kumar, Andrew S. Lesniewski, and 
    Diana E. Woodward "Managing smile risk." The Best of Wilmott 1 (2002): 249-296.
    '''
    output = np.zeros(len(K))
    
    for i in range(0,len(K)):
        if K[i] == f: #ATM equation in Managing smile risk
            part_1 = (1.0 - beta)**2.0*alpha**2.0/(24.0*f**(2.0 - 2.0*beta))
            part_2 = rho*beta*alpha*nu/(4.0*f**(1.0 - beta))
            part_3 = (2.0 - 3.0*rho**2)*nu**2.0/24.0
            
            output[i] = (alpha/f**(1 - beta))*(1 + (part_1 + part_2 + part_3)*t_exp )
        
        else:
            logfK = np.log(f/K[i])
            fkbpow = (f*K[i])**((1.0 - beta)/2.0)
            z = nu*fkbpow*logfK/alpha
            xz = np.log((np.sqrt(1.0 - 2.0*rho*z + z**2.0 ) + z - rho)/(1.0-rho))
            
            part_1 = ((1.0-beta)**2.0)*(alpha**2.0)/(24.0*fkbpow**2.0)
            part_2 = (rho*beta*nu*alpha)/(4.0*fkbpow)
            part_3 = (2.0-3.0*rho**2)*nu**2.0/24.0
            part_4 = ((1.0-beta)**2)*(logfK**2)/24.0
            part_5 = ((1.0-beta)**4)*(logfK**4)/1920.0
            
            output[i] = (alpha*z*(1 + (part_1 + part_2 + part_3)*t_exp ))/(fkbpow*xz*(1 + part_4 + part_5 ))
            
    return output

def atm_sigma_to_alpha(f,t_exp,sigma_atm,beta,nu,rho):
    '''Returns alpha given the forward price, the at-the-money volatility, the 
    time to exirpy (t_exp) and the other parameters in the SABR model by 
    solving a cubic equation for alpha, equation (2.18) in Hagan, Patrick S., 
    Deep Kumar, Andrew S. Lesniewski, and Diana E. Woodward. 
    "Managing smile risk." The Best of Wilmott 1 (2002): 249-296. 
    '''
    #The coeffceints of the polynomial we find the roots of
    p_3 = -sigma_atm
    p_2 =  (1 + (2-3*rho**2)*nu**2*t_exp/24)/f**(1.-beta)
    p_1 = rho*beta*nu*t_exp/(4*f**(2-2*beta))
    p_0 = (1-beta)**2*t_exp/(24*f**(3-3*beta))
    coeffs = [p_0,p_1,p_2,p_3]
    
    r = np.roots(coeffs)    #find the roots of the cubic equation
    
    return r[(r.imag==0) & (r.real>=0)].real.min() 

def SABR_calibration(f, t_exp, sigma_atm, beta, strikes, vols,guess):
    ''' Returns the parameters alpha, nu and rho given a parameter beta, 
    forward price, a list of market volatilities and corrsponding strike 
    spread. Instead of doing a regression in all three parameters, this method 
    calculates alpha when needed from nu and rho. Hence a regression is done 
    in only two variables.
    '''
    def func_to_optimize(K,nu,rho):
        alpha = atm_sigma_to_alpha(f,t_exp,sigma_atm,beta,nu,rho)
        return  SABR_market_vol(K,f,t_exp,alpha,beta,nu,rho)
     
    popt, pcov = minimize(func_to_optimize, strikes, vols, args = (guess[1],guess[2]), maxfev=10000)
      
    nu = popt[0]
    rho = popt[1]
    alpha = atm_sigma_to_alpha(f,t_exp,sigma_atm,beta,nu,rho)
    
    return [alpha, nu, rho]
        


#10Y30Y 2024
beta = 0.5
sigmas = np.array([0.4364, 0.4191, 0.41706,0.41605, 0.41619,0.41757,0.42003,0.42370,0.42847,0.43428,0.46603])
#sigmas = sigmas*10
strikes_in_bps = np.array([-200,-100,-75,-50,-25,0,25,50,75,100,200])
t_exp = 10
tenor = 30
atm_sigma = 0.41757
#atm_sigma =atm_sigma *10
f = 0.23401
#f = f*10
guess = [0.01, 10,-0.5]
strikes = f + strikes_in_bps*0.0001
#Calling the SABR_calibration function defined below to return the parameters.
alpha, nu, rho = SABR_calibration(f, t_exp, atm_sigma, beta, strikes, sigmas, guess)
    
print("alpha:",alpha,"rho:",rho, "nu:", nu)
Ks_in_bps = np.linspace(-200,200,60)
Ks = f + Ks_in_bps*0.0001
vols_from_Ks = SABR_market_vol(Ks,f,t_exp,alpha,beta,nu,rho)

fig, ax = plt.subplots(figsize=(14, 7))
plt.plot(strikes_in_bps, sigmas, 'x', color = 'red', label = 'Market data')
plt.plot(Ks_in_bps,vols_from_Ks, color = 'black', label= 'Estimated volatility smile')
plt.xlabel("Strike",fontsize=14)
plt.ylabel("Implied Volatility", fontsize=14)
plt.title("10Y30Y EUR Swaption Implied Volatility", fontsize=18)
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', linewidth=0.5)
#plt.savefig("/Users/nannaingemannohrt/Desktop/master_thesis/main/plots/10Y30Y_est.png")  
plt.show()
plt.close()

TypeError: minimize() got multiple values for argument 'args'